In [28]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage
import numpy as np
from scipy.ndimage import convolve
import math
from sklearn import svm
from sklearn.metrics import accuracy_score



In [34]:
# Load the dataset
(train_X, train_y), (test_X, test_y) = mnist.load_data()

train_vector = np.empty((60000 , 324))
test_vector = np.empty((10000 , 324))

In [18]:
def calculate_magnitude_and_phase(image , magnitude , phase):
    for i in range(1, len(image)-1):
        for j in range(len(image)-1):
             dy = np.subtract(image[i][j+1], image[i][j-1])
             dx = np.subtract(image[i+1][j], image[i-1][j])
             mag = np.sqrt(dy**2 + dx**2)
             magnitude[i][j] = mag
             phase[i][j] = np.degrees(np.arctan2(dy, dx))


In [40]:
def normalize_vector(block_vector):
    squared_sum = 0
    for i in range(36):
        squared_sum +=  block_vector[i] ** 2
    for i in range(36):

        if squared_sum == 0:

            block_vector[i] = 0 
        else:
            block_vector[i] = block_vector[i] / math.sqrt(squared_sum)

            # block_vector[i] = block_vector[i] / math.sqrt(squared_sum)

    return block_vector

In [41]:
def cell_loop(cell_mag , cell_phase , mag_vec):
    
    for i in range(8):
        for j in range(8):

            array_range = np.array([10 , 30 , 50 , 70 , 90 , 110 , 130 , 150 , 170])

            if cell_phase[i][j] < array_range[0] :
                mag_vec[0] = mag_vec[0] + cell_mag[i][j]

            elif cell_phase[i][j] >= array_range[8]: 
                    mag_vec[8] = mag_vec[8] + cell_mag[i][j]

            else:
                for k in range(1 , 9):
                    if array_range[k-1] <= cell_phase[i][j] < array_range[k] :
                        mag_vec[k-1] = mag_vec[k-1] + (((abs(cell_phase[i][j]-array_range[k])) / 20) * cell_mag[i][j])
                        mag_vec[k] = mag_vec[k] + (((abs(cell_phase[i][j]-array_range[k-1])) / 20) * cell_mag[i][j])



In [42]:
def Extract_Block_Vector(magnitude , phase , row , img_vector):

    index = 0

    for i in range(0 , 24 , 8):
        for j in range(0 , 24 , 8):

            block_height = 16
            block_width = 16

            start_y = i # Starting Y coordinate
            start_x = j # Starting X coordinate

            # Extract the block
            # block = train_X[0][start_y:start_y+block_height, start_x:start_x+block_width]
            block_magnitude = magnitude[start_y:start_y+block_height, start_x:start_x+block_width]
            block_phase = phase[start_y:start_y+block_height, start_x:start_x+block_width]


            cell1_magnitude = block_magnitude[0 : 0 + 8 , 0 : 0 + 8]
            cell2_magnitude = block_magnitude[0 : 0 + 8 , 8 : 8 + 8]
            cell3_magnitude = block_magnitude[8 : 8 + 8 , 0 : 0 + 8]
            cell4_magnitude = block_magnitude[8 : 8 + 8 , 8 : 8 + 8]

            cell1_phase = block_phase[0 : 0 + 8 , 0 : 0 + 8]
            cell2_phase = block_phase[0 : 8 + 8 , 8 : 8 + 8]
            cell3_phase = block_phase[8 : 8 + 8 , 0 : 0 + 8]
            cell4_phase = block_phase[8 : 8 + 8 , 8 : 8 + 8]

            
            mag_vector1 = np.zeros(9)
            cell_loop(cell1_magnitude , cell1_phase , mag_vector1)

            mag_vector2 = np.zeros(9)
            cell_loop(cell2_magnitude , cell2_phase , mag_vector2)

            mag_vector3 = np.zeros(9)
            cell_loop(cell3_magnitude , cell3_phase , mag_vector3)

            mag_vector4 = np.zeros(9)
            cell_loop(cell4_magnitude , cell4_phase , mag_vector4)

            combined_vector = np.concatenate((mag_vector1, mag_vector2, mag_vector3, mag_vector4))
            vect = normalize_vector(combined_vector)
            img_vector[row,index:index+36] = vect
            index = index +36

            if index > 324:
                index = 0

            # print(mag_vector1)
            # print("____________________________________")
            # print(mag_vector2)
            # print("____________________________________")
            # print(mag_vector3)
            # print("____________________________________")
            # print(mag_vector4)
            # print("____________________________________")
            # print(img_vector[0])

            


In [43]:
for i in range(len(train_X)):

    magnitude = np.zeros(shape=(32, 32))
    phase = np.zeros(shape=(32, 32))

    calculate_magnitude_and_phase(train_X[i] , magnitude , phase)

    magnitude = magnitude.astype(int)
    phase = phase.astype(int)

    Extract_Block_Vector(magnitude , phase , i , train_vector)

    if i == 0 :
        print(train_vector[0])
    


[0.00000000e+00 1.01357237e-02 5.33459142e-04 1.11371295e-02
 4.25831421e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 6.58372812e-01 1.53552003e-01 4.42303142e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.81891042e-02 3.00608906e-02
 4.50726181e-02 0.00000000e+00 3.36921564e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 4.21919387e-01
 2.69967762e-01 2.84446030e-01 3.31390435e-01 3.05943498e-01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.25948247e-01 1.22666679e-01 3.53338652e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 3.24262604e-01 2.20930113e-01 2.01386583e-01
 1.52826817e-01 6.93967280e-02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 3.37054869e-01 2.15666668e-01
 2.27232790e-01 2.64734835e-01 2.44406274e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 3.19799143e-01
 9.23644880e-02 1.013437

In [44]:
for i in range(len(test_X)):

    magnitude = np.zeros(shape=(32, 32))
    phase = np.zeros(shape=(32, 32))

    calculate_magnitude_and_phase(test_X[i] , magnitude , phase)

    magnitude = magnitude.astype(int)
    phase = phase.astype(int)

    Extract_Block_Vector(magnitude , phase , i , test_vector)

    if i == 0 :
        print(test_vector[0])
    


[0.1273825  0.06669476 0.03637896 0.         0.02357896 0.
 0.         0.         0.         0.28477203 0.25550886 0.1698948
 0.         0.         0.         0.         0.         0.
 0.14540356 0.01953685 0.07828775 0.02941755 0.08609126 0.
 0.         0.         0.         0.82268099 0.07198599 0.14661058
 0.1762948  0.17807726 0.         0.         0.         0.
 0.17399758 0.15611759 0.10380683 0.         0.         0.
 0.         0.         0.         0.17391182 0.         0.
 0.         0.         0.         0.         0.         0.
 0.50266347 0.04398391 0.08958002 0.10771728 0.10880637 0.
 0.         0.         0.         0.42498629 0.32257727 0.43848418
 0.22918971 0.29798264 0.         0.         0.         0.
 0.21595961 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.52773798 0.40056887 0.54449934 0.28460239 0.37002783 0.
 0.         0.         0.        

In [46]:
clf = svm.SVC(kernel='linear')
clf.fit(train_vector, train_y)
predicted_labels = clf.predict(test_vector)
accuracy = accuracy_score(test_y, predicted_labels)
print(f"Accuracy of the scratch code (HOG): {accuracy * 100}%")

Accuracy of the scratch code (HOG): 92.24%
